In [61]:
from urllib.parse import urlparse

path = './may-28.txt'
data = []
with open(path, 'r', encoding='utf8') as file:
    l = 0
    for line in file:
        data.append(line)
        if l > 100:
            break
        l += 1

class Page:
    def __init__(self, info):
        if ' ' in info:
            parts = info.split(' | ')
        else:
            parts = info, ''
        self.url, self.title = parts[0], ''.join(parts[1:])
        self.parse = urlparse(self.url)
        
class Collection:
    def __init__(self):
        self.urls = []
        
    def add(self, urls):
        if type(urls) is str:
            urls = Page(urls)
            self.urls.append(urls)
        elif type(urls) is list:
            for url in urls:
                self.urls.append(Page(url))
    
    def find(self, attr, value=None):
        if not callable(attr):
            attr = lambda x: getattr(x, attr) == value
        return list(filter(attr, self.urls))
    
c = Collection()
c.add(data)
# print(c.urls[100].parse)
c.find(lambda x: len(x.url)>1000)[5]

In [35]:
data[1][5]

'e'